In [16]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\exploitation'

In [3]:
data_f = pd.DataFrame(columns=['user_id','sig_id','fakeness'])
visual_f = pd.DataFrame()

In [4]:
datalist = os.listdir(data_folder)

In [7]:
for data in tqdm.tqdm(datalist):
    if not(data.endswith('.mat')): continue
    if (data.split('.')[0].split('_')[0] == 'forg'): fakeness = True
    else: fakeness = False
    user_id = data.split('.')[0].split('_')[-1]
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        data_f = data_f.append(pd.DataFrame([[user_id, row+1, fakeness]], columns=['user_id', 'sig_id', 'fakeness']))
        visual_f = visual_f.append(pd.DataFrame(np.expand_dims(matfile['features'][row], 1).T))

100%|████████████████████████████████████████████████████████████████████████████████| 602/602 [08:42<00:00,  1.15it/s]


In [8]:
data_f = data_f.reset_index().drop(columns=['index'])
visual_f = visual_f.reset_index().drop(columns=['index'])

In [9]:
visual_f.shape

(16105, 2048)

In [10]:
data_f.shape

(16105, 3)

In [11]:
data_f.to_csv(os.path.join(data_folder,'data_features.csv'), index=False)
visual_f.to_csv(os.path.join(data_folder,'visual_features.csv'), index=False)

In [20]:
data_f.loc[data_f['fakeness']==False]

,user_id,sig_id,fakeness
8905,100,1,False
8906,100,2,False
8907,100,3,False
8908,100,4,False
8909,100,5,False
8910,100,6,False
8911,100,7,False
8912,100,8,False
8913,100,9,False
8914,100,10,False


In [13]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_14 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,15))]
data_f_gen_valid_10 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(15,25))]

In [19]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique()):
    y_train = data_f['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2,predictor='gpu_predictor', scale_pos_weight=8946/14)
    clf = SVC(class_weight='balanced', gamma='scale')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid,y_valid)
    print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_10.loc[data_f_gen_valid_10['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid,y_valid)
    print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_14.loc[choice(data_f_gen_14.loc[data_f_gen_14['user_id']!=3].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid,y_valid)
    print(score)
    scores_sum += score
    run_count += 1
    print(scores_sum/run_count)
print(scores_sum/run_count)



  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

0.0
1.0
1.0
0.6666666666666666




  0%|▎                                                                               | 1/300 [00:14<1:10:23, 14.13s/it]

0.0
1.0
1.0
0.6666666666666666




  1%|▌                                                                               | 2/300 [00:22<1:00:55, 12.27s/it]

0.0
1.0
1.0
0.6666666666666666




  1%|▊                                                                                 | 3/300 [00:31<56:31, 11.42s/it]

0.0
1.0
1.0
0.6666666666666666




  1%|█                                                                                 | 4/300 [00:41<53:53, 10.92s/it]

KeyboardInterrupt: 

In [21]:
y_valid_fakes

,user_id,sig_id,fakeness
90,103,1,True
91,103,2,True
92,103,3,True
93,103,4,True
94,103,5,True
95,103,6,True
96,103,7,True
97,103,8,True
98,103,9,True
99,103,10,True
